In [148]:
'''
在获得粗略数据后，构建可以被模型读取的数据
'''
# 截取的时段为患者开始接收RRT到结束的时段
# 计算每个时刻及撤机7d内的reward
# 动作要量化为IHD、CRRT、No的片段
# 状态进行高斯补全
# 考虑到动作可能重叠的情况，则IHD为优先级

'\n在获得粗略数据后，构建可以被模型读取的数据\n'

In [149]:
TIMEDURING = 6
CON_WINDOWS = 7 * 24 // TIMEDURING

# 参数
C_1 = 0.3
C_2 = 0.8
C_3 = -0.1
C4 = 0.8
C5 = 0.2
k = -0.1


import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, WhiteKernel


In [150]:

# file_demo = pd.read_csv('./data/temp/30166100.csv')

# df = file_demo
ID_col = ['stay_id', 'step']
state_col =    ['ph',
 'po2',
 'calcium',
 'bicarbonate',
 'creatinine',
 'sodium',
 'potassium',
 'uo_rt_24hr']
col_list = ['eGFR'] + state_col



In [158]:
# 首先利用高斯处理过程处理各个state、action值
# 选择用于高斯过程的特征（这里使用时间的浮点数表示）
df_merge = pd.DataFrame()
file_pool = os.listdir('./data/temp/')
for file in tqdm(file_pool):
    df = pd.read_csv('./data/temp/' + file)

    df['start_time'] = pd.to_datetime(df['start_time'])
    df['end_time'] = pd.to_datetime(df['end_time'])
    # 按照 start_time 升序排序
    df.sort_values('start_time', inplace=True)
    # 重置索引，并创建新的 step 列作为排序后的序号
    df.reset_index(drop=True, inplace=True)
    df['step'] = df.index  # index 是自动从0开始的整数序列
    
    
    #使用均匀分布的形式进行拟合
    # 设置step为索引
    df.set_index('step', inplace=True)
    # 对每列进行线性插值
    df['eGFR'] = df['avg_eGFR']
    df_process = df[col_list]
    df_process = df_process.interpolate(method='linear')

    # 填充插值后仍存在的缺失值
    df_process.fillna(method='ffill', inplace=True)  # 先前无存在值，使用最近的存在值
    df_process.fillna(method='bfill', inplace=True)  # 之后无存在值，也使用最近的存在值

    df[col_list] = df_process


    # 对数据的长度进行裁切
    # 寻找第一个和最后一个IHD、CRRT记录点
    # 计算每列的第一个和最后一个有效索引
    first_index_ihd = df['ihd'].first_valid_index()
    first_index_crrt = df['crrt'].first_valid_index()
    last_index_ihd = df['ihd'].last_valid_index()
    last_index_crrt = df['crrt'].last_valid_index()

    # 使用min和max来确定第一个和最后一个有效索引，处理None的情况
    first_index = min(first_index_ihd if first_index_ihd is not None else float('inf'),
                    first_index_crrt if first_index_crrt is not None else float('inf'))

    last_index = max(last_index_ihd if last_index_ihd is not None else float('-inf'),
                    last_index_crrt if last_index_crrt is not None else float('-inf'))
    # 需要检查是否有无效的索引值（如果两个列都是None）
    first_index = None if first_index == float('inf') else first_index
    last_index = None if last_index == float('-inf') else last_index

    if (first_index is None) or (last_index is None):
        print(f'{file} have no rrt record')
        continue
    # print(f"first non-null value at step: {first_index}")
    # print(f"last non-null value at step: {last_index}")
    # 计算eGFR的末期值
    max_index = min(last_index + CON_WINDOWS, df.index.max())
    # 使用loc来安全地获取范围内的数据并计算最小值
    min_value = df.loc[last_index:max_index, 'eGFR'].min()

    df = df.loc[first_index:last_index+1]
    df.reset_index(drop=True, inplace=True)
    df['step'] = df.index  # index 是自动从0开始的整数序列

    # 对动作进行处理
    df['action'] = np.where(pd.notna(df['crrt']), 2, 
                            np.where(pd.notna(df['ihd']), 1, 0)) #若crrt非空则2，若IHD非空则1，否则0

    # 计算sgn(eGFR >= 90)
    df['sgn_eGFR'] = (df['eGFR'] >= 90).astype(int)

    # 计算ΔeGFR
    df['delta_eGFR'] = df['eGFR'].diff().fillna(0)  # 填充第一个值为0或其他适当方法

    # 计算第二项中的表达式
    df['exp_part_2'] = 2 / (1 + np.exp(-df['delta_eGFR'])) - 1

    # 计算第三项中的表达式
    df['exp_part_3'] = 1 - np.exp(k * np.maximum(0, df['step'] - 14 * 24 / TIMEDURING))

    # 计算最终reward
    df['reward'] = C_1 * df['sgn_eGFR'] + C_2 * df['exp_part_2'] + C_3 * df['exp_part_3']



    # 对于终末期的reward进行计算
    df.loc[df.index[-1], 'reward'] = C4 *  (1 if min_value > 90 else -1) + C5 * df.loc[df.index[-1], 'delta_eGFR']# 参数
    # df_merge = pd.concat([df_merge, df[ID_col+state_col+['action', 'reward']]])
    df_merge = pd.concat([df_merge, df])
df_merge.to_csv('./data/res.csv')


# # 将 starttime 和 endtime 转换为浮点数（UNIX 时间戳形式），然后计算平均值
# # df['time_float'] = (df['start_time'].astype('int64') + df['end_time'].astype('int64')) / 2 / 10**9
# df['eGFR'] = df['avg_eGFR']
# col = 'eGFR'
# # 分离出已知和未知的数据点
# known_data = df.dropna(subset=[col])
# unknown_data = df[df[col].isna()]

# # 定义高斯过程回归模型
# kernel = 1.0 * RBF(length_scale=10.0) + WhiteKernel(noise_level=0.1)
# gp = GaussianProcessRegressor(kernel=kernel, optimizer='fmin_l_bfgs_b', n_restarts_optimizer=10)

# # 训练高斯过程回归器
# gp.fit(known_data[['step']], known_data[col])

# # 使用模型对未知数据进行预测
# x_pred, sigma = gp.predict(unknown_data[['step']], return_std=True)

# df.loc[df[col].isna(), 'eGFR'] = x_pred
# # eGFR要保留原值


  0%|          | 0/3453 [00:00<?, ?it/s]

  1%|          | 18/3453 [00:00<00:39, 85.96it/s]

30034929.csv have no rrt record


  3%|▎         | 88/3453 [00:00<00:37, 89.28it/s]

30215388.csv have no rrt record


...

 93%|█████████▎| 3223/3453 [00:59<00:06, 38.32it/s]

39210893.csv have no rrt record


 96%|█████████▌| 3320/3453 [01:02<00:03, 38.02it/s]

39554910.csv have no rrt record
39568034.csv have no rrt record


100%|██████████| 3453/3453 [01:06<00:00, 52.13it/s]


In [157]:
df_merge.to_csv('./data/res.csv')

In [152]:
# 对数据的长度进行裁切
# 寻找第一个和最后一个IHD、CRRT记录点

# first_index = min(df['ihd'].first_valid_index(), df['crrt'].first_valid_index())
# last_index = max(df['ihd'].last_valid_index(), df['crrt'].last_valid_index())
# print(f"first non-null value at step: {first_index}")
# print(f"last non-null value at step: {last_index}")
# # 计算eGFR的末期值
# max_index = min(last_index + CON_WINDOWS, df.index.max())
# # 使用loc来安全地获取范围内的数据并计算最小值
# min_value = df.loc[last_index:max_index, 'eGFR'].min()

# df = df.loc[first_index:last_index+1]
# df.reset_index(drop=True, inplace=True)
# df['step'] = df.index  # index 是自动从0开始的整数序列

In [153]:
# # 对动作进行处理
# df['action'] = np.where(pd.notna(df['crrt']), 2, 
#                         np.where(pd.notna(df['ihd']), 1, 0)) #若crrt非空则2，若IHD非空则1，否则0


In [154]:
# # 参数
# C_1 = 0.3
# C_2 = 0.8
# C_3 = -0.1
# C4 = 0.8
# C5 = 0.2
# k = -0.1

# # 计算sgn(eGFR >= 90)
# df['sgn_eGFR'] = (df['eGFR'] >= 90).astype(int)

# # 计算ΔeGFR
# df['delta_eGFR'] = df['eGFR'].diff().fillna(0)  # 填充第一个值为0或其他适当方法

# # 计算第二项中的表达式
# df['exp_part_2'] = 2 / (1 + np.exp(-df['delta_eGFR'])) - 1

# # 计算第三项中的表达式
# df['exp_part_3'] = 1 - np.exp(k * np.maximum(0, df['step'] - 14 * 24 / TIMEDURING))

# # 计算最终reward
# df['reward'] = C_1 * df['sgn_eGFR'] + C_2 * df['exp_part_2'] + C_3 * df['exp_part_3']




In [155]:


# 对于终末期的reward进行计算
df.loc[df.index[-1], 'reward'] = C4 *  (1 if min_value > 90 else -1) + C5 * df.loc[df.index[-1], 'delta_eGFR']# 参数
df[ID_col+state_col+['action', 'reward']].to_csv('./data/res.csv')
